In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load Dataset
# -----------------------------
data_dir = 'path_to_dataset'  # dataset folder containing subfolders for 3 classes
desired_width, desired_height = 224, 224

def load_dataset(data_dir, img_size=(224,224)):
    classes = os.listdir(data_dir)
    images, labels = [], []
    for class_idx, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)
            img = img.astype(np.float32) / 255.0
            images.append(img)
            labels.append(class_idx)
    images = np.array(images)
    labels = np.array(labels)
    # shuffle
    idx = np.arange(len(images))
    np.random.shuffle(idx)
    return images[idx], labels[idx], classes

x_data, y_data, classes = load_dataset(data_dir)

# Split into train/test
train_split = int(0.8 * len(x_data))
x_train, y_train = x_data[:train_split], y_data[:train_split]
x_test, y_test = x_data[train_split:], y_data[train_split:]

# -----------------------------

In [ ]:
# Build Siamese Network (Embedding Model)
# -----------------------------
input_shape = (desired_height, desired_width, 3)

def create_base_cnn(input_shape):
    inp = Input(shape=input_shape)
    x = Conv2D(32, (3,3), activation='relu')(inp)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    model = Model(inp, x, name='base_cnn')
    return model

base_cnn = create_base_cnn(input_shape)

# Inputs for Siamese
input_left = Input(shape=input_shape)
input_right = Input(shape=input_shape)

# Get embeddings
embedding_left = base_cnn(input_left)
embedding_right = base_cnn(input_right)

# Contrastive Loss Distance
def euclidean_distance(vects):
    x, y = vects
    return tf.sqrt(tf.reduce_sum(tf.square(x - y), axis=1, keepdims=True))

distance = tf.keras.layers.Lambda(euclidean_distance)([embedding_left, embedding_right])

# Siamese Model
siamese_model = Model(inputs=[input_left, input_right], outputs=distance)


In [ ]:
# Prepare Pairs for Training
# -----------------------------
def create_pairs(x, y):
    pairs, labels = [], []
    num_classes = len(np.unique(y))
    class_idx = [np.where(y==i)[0] for i in range(num_classes)]
    for c in range(num_classes):
        for i in range(len(class_idx[c])):
            for j in range(i+1, len(class_idx[c])):
                # positive pair
                pairs += [[x[class_idx[c][i]], x[class_idx[c][j]]]]
                labels += [1]
                # negative pair
                neg_c = (c + np.random.randint(1,num_classes)) % num_classes
                neg_idx = np.random.choice(class_idx[neg_c])
                pairs += [[x[class_idx[c][i]], x[neg_idx]]]
                labels += [0]
    return np.array(pairs), np.array(labels)

pairs_train, labels_train = create_pairs(x_train, y_train)


In [ ]:
# Compile Siamese Model
# -----------------------------
def contrastive_loss(y_true, y_pred, margin=1.0):
    y_true = tf.cast(y_true, y_pred.dtype)
    return tf.reduce_mean(y_true * tf.square(y_pred) + (1 - y_true) * tf.square(tf.maximum(margin - y_pred, 0)))

siamese_model.compile(optimizer='adam', loss=contrastive_loss, metrics=['accuracy'])
siamese_model.summary()


In [ ]:
# Train Siamese Model
# -----------------------------
epochs = 50
batch_size = 16

siamese_model.fit([pairs_train[:,0], pairs_train[:,1]], labels_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1)


In [ ]:
# Build Embeddings for Training Set
# -----------------------------
train_embeddings = base_cnn.predict(x_train)
test_embeddings = base_cnn.predict(x_test)


In [ ]:
# Nearest Neighbor Classification
# -----------------------------
def predict_class(test_emb, train_emb, train_labels):
    preds = []
    for emb in test_emb:
        distances = np.linalg.norm(train_emb - emb, axis=1)
        closest_idx = np.argmin(distances)
        preds.append(train_labels[closest_idx])
    return np.array(preds)

y_pred = predict_class(test_embeddings, train_embeddings, y_train)


In [ ]:
# Evaluate
# -----------------------------
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(conf_matrix, annot=True, fmt='d', xticklabels=classes, yticklabels=classes)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
